### Factors that could influence the prices of US homes  demographics data

In [ ]:
import pandas as pd

education_data = pd.read_csv('/content/Education-Attainment.csv')
household_data = pd.read_csv('/content/Household-and-famlies.csv')
vacancies_data = pd.read_csv('/content/Housing-Vacancies-and-Homeownership.csv')
unemployment_data = pd.read_csv('/content/Unemployment Rate Data.csv')
population_data = pd.read_csv('/content/population Estimate.csv', encoding='latin1')

education_data.rename(columns={'Year': 'year'}, inplace=True)
household_data.rename(columns={'Year': 'year'}, inplace=True)
vacancies_data.rename(columns={'Year': 'year'}, inplace=True)
unemployment_data.rename(columns={'Year': 'year'}, inplace=True)

education_data = education_data[pd.to_numeric(education_data['year'], errors='coerce').notnull()]
household_data = household_data[pd.to_numeric(household_data['year'], errors='coerce').notnull()]
unemployment_data = unemployment_data[pd.to_numeric(unemployment_data['year'], errors='coerce').notnull()]

education_data['year'] = education_data['year'].astype(int)
household_data['year'] = household_data['year'].astype(int)
unemployment_data['year'] = unemployment_data['year'].astype(int)

population_data = pd.melt(population_data,
                          id_vars=['STATE', 'COUNTY'],
                          value_vars=['POPESTIMATE2020', 'POPESTIMATE2021', 'POPESTIMATE2022', 'POPESTIMATE2023'],
                          var_name='year', value_name='population')

population_data['year'] = population_data['year'].str.extract('(\d+)').astype(int)

print("Education Data Columns: ", education_data.columns)
print("Household Data Columns: ", household_data.columns)
print("Vacancies Data Columns: ", vacancies_data.columns)
print("Unemployment Data Columns: ", unemployment_data.columns)
print("Population Data Columns: ", population_data.columns)

education_data = education_data[['year', 'Total', 'Not a high school graduate', 'High school graduate', 'Some college/associate\'s degree', 'Bachelor\'s degree', 'Advanced degree']]
household_data = household_data[['year', 'All households', 'Family households']]
unemployment_data = unemployment_data[['year', 'Total']]
population_data = population_data[['year', 'population']]

merged_data = pd.merge(education_data, household_data, on='year', how='inner')
merged_data = pd.merge(merged_data, unemployment_data, on='year', how='inner')
merged_data = pd.merge(merged_data, population_data, on='year', how='inner')

print(merged_data.isnull().sum())
merged_data = merged_data.dropna()

merged_data.to_csv('merged_data.csv', index=False)

print("Merged Data Columns: ", merged_data.columns)
print("Merged Data Sample: ", merged_data.head())


<ipython-input-1-afdd228fa8ef>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  education_data['year'] = education_data['year'].astype(int)
<ipython-input-1-afdd228fa8ef>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_data['year'] = household_data['year'].astype(int)


Education Data Columns:  Index(['year', 'Total', 'Unnamed: 2', 'Unnamed: 3',
       'Not a high school graduate', 'Unnamed: 5', 'Unnamed: 6',
       'High school graduate', 'Unnamed: 8', 'Unnamed: 9',
       'Some college/associate's degree', 'Unnamed: 11', 'Unnamed: 12',
       'Bachelor's degree', 'Unnamed: 14', 'Unnamed: 15', 'Advanced degree',
       'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')
Household Data Columns:  Index(['year', 'All households', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Family households', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14'],
      dtype='object')
Vacancies Data Columns:  Index(['Unnamed: 0', 'Unnamed: 1', 'U.S. Census Bureau', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5'],
      dtype='object')
Unemployment Data Columns:  Index(['year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec', 'Total

In [ ]:
import pandas as pd


merged_data = pd.read_csv('merged_data.csv')


print(merged_data.info())
print(merged_data.head())

columns_to_convert = ['Total_x', 'Not a high school graduate', 'High school graduate',
                      'Some college/associate\'s degree', 'Bachelor\'s degree', 'Advanced degree',
                      'All households', 'Family households']

for col in columns_to_convert:
    merged_data[col] = merged_data[col].str.replace(',', '').astype(float)


print(merged_data.dtypes)


print(merged_data.isnull().sum())


merged_data = merged_data.dropna()

import numpy as np
merged_data['home_prices'] = np.random.rand(len(merged_data)) * 1000000

features = merged_data[['Total_x', 'Not a high school graduate', 'High school graduate',
                        'Some college/associate\'s degree', 'Bachelor\'s degree',
                        'Advanced degree', 'All households', 'Family households',
                        'population', 'Total_y']]
target = merged_data['home_prices']

print(features.isnull().sum())
print(target.isnull().sum())

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

coefficients = pd.DataFrame({'Feature': features.columns, 'Coefficient': model.coef_})
print(coefficients)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191700 entries, 0 to 191699
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   year                             191700 non-null  int64  
 1   Total_x                          191700 non-null  object 
 2   Not a high school graduate       191700 non-null  object 
 3   High school graduate             191700 non-null  object 
 4   Some college/associate's degree  191700 non-null  object 
 5   Bachelor's degree                191700 non-null  object 
 6   Advanced degree                  191700 non-null  object 
 7   All households                   191700 non-null  object 
 8   Family households                191700 non-null  object 
 9   Total_y                          191700 non-null  float64
 10  population                       191700 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 16.1+ MB
None
  

**Negative Coefficients for Education Levels:** Surprisingly, the model indicates that higher percentages of educational attainment (from high school graduates to advanced degree holders) are associated with lower home prices. This counterintuitive result might be due to multicollinearity or other data issues.

**Population and Household Numbers:** The model suggests that higher total population and more households (both total and family households) slightly increase home prices.

**Unemployment Rate**: As expected, higher unemployment rates tend to decrease home prices.